In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
start = time.time()

In [2]:
df = spark.createDataFrame([
    ('client-uuid-1',  'cat-1, cat-2, cat-3'),
    ('client-uuid-2',  'cat-1, cat-4, cat-5'),
    ('client-uuid-3',  'cat-6, cat-7'),
    ('client-uuid-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('client-uuid-5',  'cat-8, cat-10'),
    ('client-uuid-6',  'cat-1, cat-9, cat-10'),
    ('client-uuid-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('client-uuid-8',  'cat-7, cat-9'),
    ('client-uuid-9',  'cat-1'),
    ('client-uuid-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])


In [3]:
df2 = spark.createDataFrame([
    ('client-uuid-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('client-uuid-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('client-uuid-3',  'cat-4, cat-14, cat-15'),
    ('client-uuid-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('client-uuid-5',  'cat-8, cat-10, cat-11'),
    ('client-uuid-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('client-uuid-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('client-uuid-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('client-uuid-9',  'cat-2'),
    ('client-uuid-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
print(time.time() - start)

1.8494501113891602

In [5]:
df.show(10,False)

+--------------+--------------------------------------------------------------+
id_cliente |categorias |
+--------------+--------------------------------------------------------------+
client-uuid-1 |cat-1, cat-2, cat-3 |
client-uuid-2 |cat-1, cat-4, cat-5 |
client-uuid-3 |cat-6, cat-7 |
client-uuid-4 |cat-1, cat-2, cat-7, cat-10 |
client-uuid-5 |cat-8, cat-10 |
client-uuid-6 |cat-1, cat-9, cat-10 |
client-uuid-7 |cat-1, cat-4, cat-5, cat-10 |
client-uuid-8 |cat-7, cat-9 |
client-uuid-9 |cat-1 |
client-uuid-10|cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10|
+--------------+--------------------------------------------------------------+

In [6]:
df.printSchema()

root
-- id_cliente: string (nullable = true)
-- categorias: string (nullable = true)

In [7]:
dfteste = df.withColumn(
    "col3",
    split(regexp_replace("categorias", r"(^\[\[\[)|(\]\]\]$)", ""), ", ")
)
dfteste.show(10, False)

+--------------+--------------------------------------------------------------+----------------------------------------------------------------+
id_cliente |categorias |col3 |
+--------------+--------------------------------------------------------------+----------------------------------------------------------------+
client-uuid-1 |cat-1, cat-2, cat-3 |[cat-1, cat-2, cat-3] |
client-uuid-2 |cat-1, cat-4, cat-5 |[cat-1, cat-4, cat-5] |
client-uuid-3 |cat-6, cat-7 |[cat-6, cat-7] |
client-uuid-4 |cat-1, cat-2, cat-7, cat-10 |[cat-1, cat-2, cat-7, cat-10] |
client-uuid-5 |cat-8, cat-10 |[cat-8, cat-10] |
client-uuid-6 |cat-1, cat-9, cat-10 |[cat-1, cat-9, cat-10] |
client-uuid-7 |cat-1, cat-4, cat-5, cat-10 |[cat-1, cat-4, cat-5, cat-10] |
client-uuid-8 |cat-7, cat-9 |[cat-7, cat-9] |
client-uuid-9 |cat-1 |[cat-1] |
client-uuid-10|cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10|[cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10]|
+--------------+--------------------------------------------------------------+----------------------------------------------------------------+

In [8]:
dfteste = dfteste.withColumn("col3", col("col3").cast("array<string>"))
dfteste.printSchema()

root
-- id_cliente: string (nullable = true)
-- categorias: string (nullable = true)
-- col3: array (nullable = true)
 |-- element: string (containsNull = true)

In [9]:
dfnew = dfteste.drop('categorias')
dfnew = dfnew.withColumnRenamed('col3', 'categorias')

In [10]:
dfnew.show(10,False)

+--------------+----------------------------------------------------------------+
id_cliente |categorias |
+--------------+----------------------------------------------------------------+
client-uuid-1 |[cat-1, cat-2, cat-3] |
client-uuid-2 |[cat-1, cat-4, cat-5] |
client-uuid-3 |[cat-6, cat-7] |
client-uuid-4 |[cat-1, cat-2, cat-7, cat-10] |
client-uuid-5 |[cat-8, cat-10] |
client-uuid-6 |[cat-1, cat-9, cat-10] |
client-uuid-7 |[cat-1, cat-4, cat-5, cat-10] |
client-uuid-8 |[cat-7, cat-9] |
client-uuid-9 |[cat-1] |
client-uuid-10|[cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10]|
+--------------+----------------------------------------------------------------+

In [11]:
dfcat = dfnew.select('categorias')

In [12]:
dfcat.printSchema()

root
-- categorias: array (nullable = true)
 |-- element: string (containsNull = true)

In [13]:
arr_size = 10

In [14]:
dfcat1 = dfnew.select(['id_cliente', 'categorias']+[expr('categorias[' + str(x) + ']') for x in range(0, arr_size)])

In [15]:
dfcat1.show()

+--------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
 id_cliente| categorias|categorias[0]|categorias[1]|categorias[2]|categorias[3]|categorias[4]|categorias[5]|categorias[6]|categorias[7]|categorias[8]|categorias[9]|
+--------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
 client-uuid-1|[cat-1, cat-2, ca...| cat-1| cat-2| cat-3| null| null| null| null| null| null| null|
 client-uuid-2|[cat-1, cat-4, ca...| cat-1| cat-4| cat-5| null| null| null| null| null| null| null|
 client-uuid-3| [cat-6, cat-7]| cat-6| cat-7| null| null| null| null| null| null| null| null|
 client-uuid-4|[cat-1, cat-2, ca...| cat-1| cat-2| cat-7| cat-10| null| null| null| null| null| null|
 client-uuid-5| [cat-8, cat-10]| cat-8| cat-10| null| null| null| null| null| null| null| null|
 client-uuid-6|[cat-1, cat-9, ca...| cat-1| cat-9| cat-10| null| null| null| null| null| null| null|
 client-uuid-7|[cat-1, cat-4, ca...| cat-1| cat-4| cat-5| cat-10| null| null| null| null| null| null|
 client-uuid-8| [cat-7, cat-9]| cat-7| cat-9| null| null| null| null| null| null| null| null|
 client-uuid-9| [cat-1]| cat-1| null| null| null| null| null| null| null| null| null|
client-uuid-10|[cat-1, cat-2, ca...| cat-1| cat-2| cat-3| cat-4| cat-5| cat-6| cat-7| cat-8| cat-10| null|
+--------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+

In [16]:
new_colnames = ['id_cliente', 'categorias'] + ['cat_' + str(i) for i in range(0, arr_size)] 
df10 = dfcat1.toDF(*new_colnames)

In [17]:
df10.show(10,False)

+--------------+----------------------------------------------------------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
id_cliente |categorias |cat_0|cat_1 |cat_2 |cat_3 |cat_4|cat_5|cat_6|cat_7|cat_8 |cat_9|
+--------------+----------------------------------------------------------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
client-uuid-1 |[cat-1, cat-2, cat-3] |cat-1|cat-2 |cat-3 |null |null |null |null |null |null |null |
client-uuid-2 |[cat-1, cat-4, cat-5] |cat-1|cat-4 |cat-5 |null |null |null |null |null |null |null |
client-uuid-3 |[cat-6, cat-7] |cat-6|cat-7 |null |null |null |null |null |null |null |null |
client-uuid-4 |[cat-1, cat-2, cat-7, cat-10] |cat-1|cat-2 |cat-7 |cat-10|null |null |null |null |null |null |
client-uuid-5 |[cat-8, cat-10] |cat-8|cat-10|null |null |null |null |null |null |null |null |
client-uuid-6 |[cat-1, cat-9, cat-10] |cat-1|cat-9 |cat-10|null |null |null |null |null |null |null |
client-uuid-7 |[cat-1, cat-4, cat-5, cat-10] |cat-1|cat-4 |cat-5 |cat-10|null |null |null |null |null |null |
client-uuid-8 |[cat-7, cat-9] |cat-7|cat-9 |null |null |null |null |null |null |null |null |
client-uuid-9 |[cat-1] |cat-1|null |null |null |null |null |null |null |null |null |
client-uuid-10|[cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10]|cat-1|cat-2 |cat-3 |cat-4 |cat-5|cat-6|cat-7|cat-8|cat-10|null |
+--------------+----------------------------------------------------------------+-----+------+------+------+-----+-----+-----+-----+------+-----+

In [18]:
dfdaniel = df10

In [19]:
dfdaniel = dfdaniel.drop('categorias')
dfdaniel.show()

+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
 id_cliente|cat_0| cat_1| cat_2| cat_3|cat_4|cat_5|cat_6|cat_7| cat_8|cat_9|
+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
 client-uuid-1|cat-1| cat-2| cat-3| null| null| null| null| null| null| null|
 client-uuid-2|cat-1| cat-4| cat-5| null| null| null| null| null| null| null|
 client-uuid-3|cat-6| cat-7| null| null| null| null| null| null| null| null|
 client-uuid-4|cat-1| cat-2| cat-7|cat-10| null| null| null| null| null| null|
 client-uuid-5|cat-8|cat-10| null| null| null| null| null| null| null| null|
 client-uuid-6|cat-1| cat-9|cat-10| null| null| null| null| null| null| null|
 client-uuid-7|cat-1| cat-4| cat-5|cat-10| null| null| null| null| null| null|
 client-uuid-8|cat-7| cat-9| null| null| null| null| null| null| null| null|
 client-uuid-9|cat-1| null| null| null| null| null| null| null| null| null|
client-uuid-10|cat-1| cat-2| cat-3| cat-4|cat-5|cat-6|cat-7|cat-8|cat-10| null|
+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+

In [20]:
dfdaniel = dfdaniel.distinct()
dfdaniel.show()

+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
 id_cliente|cat_0| cat_1| cat_2| cat_3|cat_4|cat_5|cat_6|cat_7| cat_8|cat_9|
+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+
 client-uuid-1|cat-1| cat-2| cat-3| null| null| null| null| null| null| null|
 client-uuid-7|cat-1| cat-4| cat-5|cat-10| null| null| null| null| null| null|
 client-uuid-8|cat-7| cat-9| null| null| null| null| null| null| null| null|
 client-uuid-9|cat-1| null| null| null| null| null| null| null| null| null|
 client-uuid-5|cat-8|cat-10| null| null| null| null| null| null| null| null|
 client-uuid-3|cat-6| cat-7| null| null| null| null| null| null| null| null|
 client-uuid-2|cat-1| cat-4| cat-5| null| null| null| null| null| null| null|
 client-uuid-4|cat-1| cat-2| cat-7|cat-10| null| null| null| null| null| null|
 client-uuid-6|cat-1| cat-9|cat-10| null| null| null| null| null| null| null|
client-uuid-10|cat-1| cat-2| cat-3| cat-4|cat-5|cat-6|cat-7|cat-8|cat-10| null|
+--------------+-----+------+------+------+-----+-----+-----+-----+------+-----+

In [21]:
pivoted = df10.groupBy("categorias").pivot("id_cliente").agg(lit(1))
pivoted.na.fill(0).show()

+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
 categorias|client-uuid-1|client-uuid-10|client-uuid-2|client-uuid-3|client-uuid-4|client-uuid-5|client-uuid-6|client-uuid-7|client-uuid-8|client-uuid-9|
+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
[cat-1, cat-2, ca...| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 [cat-6, cat-7]| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
[cat-1, cat-4, ca...| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0|
 [cat-8, cat-10]| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
[cat-1, cat-4, ca...| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0|
[cat-1, cat-2, ca...| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
[cat-1, cat-2, ca...| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 [cat-1]| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
[cat-1, cat-9, ca...| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 [cat-7, cat-9]| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0|
+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+

In [22]:
pivoted = df10.groupBy("categorias").pivot("id_cliente").agg(lit(1))
pivoted.na.fill(0).show()

+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
 categorias|client-uuid-1|client-uuid-10|client-uuid-2|client-uuid-3|client-uuid-4|client-uuid-5|client-uuid-6|client-uuid-7|client-uuid-8|client-uuid-9|
+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
[cat-1, cat-2, ca...| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 [cat-6, cat-7]| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
[cat-1, cat-4, ca...| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0|
 [cat-8, cat-10]| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
[cat-1, cat-4, ca...| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0|
[cat-1, cat-2, ca...| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
[cat-1, cat-2, ca...| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 [cat-1]| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
[cat-1, cat-9, ca...| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 [cat-7, cat-9]| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0|
+--------------------+-------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+

In [23]:
df10.show(3,False)

+-------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
id_cliente |categorias |cat_0|cat_1|cat_2|cat_3|cat_4|cat_5|cat_6|cat_7|cat_8|cat_9|
+-------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
client-uuid-1|[cat-1, cat-2, cat-3]|cat-1|cat-2|cat-3|null |null |null |null |null |null |null |
client-uuid-2|[cat-1, cat-4, cat-5]|cat-1|cat-4|cat-5|null |null |null |null |null |null |null |
client-uuid-3|[cat-6, cat-7] |cat-6|cat-7|null |null |null |null |null |null |null |null |
+-------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
only showing top 3 rows

In [24]:
dfteste = df10.select('categorias')
dfteste.show(10,False)

+----------------------------------------------------------------+
categorias |
+----------------------------------------------------------------+
[cat-1, cat-2, cat-3] |
[cat-1, cat-4, cat-5] |
[cat-6, cat-7] |
[cat-1, cat-2, cat-7, cat-10] |
[cat-8, cat-10] |
[cat-1, cat-9, cat-10] |
[cat-1, cat-4, cat-5, cat-10] |
[cat-7, cat-9] |
[cat-1] |
[cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10]|
+----------------------------------------------------------------+

In [25]:
len(list(df10(*l)))

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3770358467880047> in <module> 
 ----> 1 len ( list ( df10 ( * l ) ) ) 

 TypeError : 'DataFrame' object is not callable

In [26]:
from itertools import chain
l = [[1,2,3],[1,2,3]]
